<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Best_keyword_extraction_clutering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia
!pip install nltk

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=9c5f5b97527f505ddeb66a30bf2660467c4e60060b1106e24e72b88051417fef
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import wikipedia
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from bs4 import BeautifulSoup
import re
from sentence_transformers import SentenceTransformer, util
import  torch
import  numpy as np

nltk.download('punkt') # Download Punkt Sentence Tokenizer if you haven't already


def fetch_document(page_title):
    """Fetches a document from Wikipedia."""
    try:
        page = wikipedia.page(page_title)
        return page.content
    except wikipedia.exceptions.PageError:
        return "Page not found."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Disambiguation error: {e.options}"


def clean_text(text):
    """Cleans the text by removing HTML tags, non-alphanumeric characters, and lowercasing."""
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text()
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', clean_text)  #Remove punctuation
    clean_text = clean_text.lower()
    return clean_text


def split_into_sentences(text):
    """Splits the text into sentences."""
    return sent_tokenize(text)


def tokenize_ngram(text, n=2):
    """Tokenizes the text using n-grams."""
    tokens = word_tokenize(text)
    ngrams = list(nltk.ngrams(tokens, n))
    return [" ".join(gram) for gram in ngrams]


def max_sum_similarity(sentences, top_n=5):
  """
  Extracts keywords using Max Sum Similarity.  This is a simplified version;
  a more robust implementation would involve more sophisticated similarity metrics
  and handling of sentence weighting.
  """
  model = SentenceTransformer('all-mpnet-base-v2') # You can experiment with other models
  embeddings = model.encode(sentences)
  similarities = util.cos_sim(embeddings, embeddings)

  scores = similarities.sum(axis=0)
  score_np = scores.cpu().numpy() #Get indices of top N sentences

  top_indices_np = np.flip (score_np.argsort()[-top_n :])

  top_indices = top_indices_np.tolist()

  keywords = [sentences[i] for i in top_indices]
  return keywords



# Example usage:
page_title = "Amitabh Bachan"  #Change to your desired Wikipedia page
document = fetch_document(page_title)

if document != "Page not found." :
    clean_doc = clean_text(document)
    sentences = split_into_sentences(clean_doc)

    #Only get bigrams from the first 10 sentences to keep processing time reasonable
    bigrams_all = []
    for sent in sentences[:100]:
      bigrams_all.extend(tokenize_ngram(sent))

    keywords = max_sum_similarity(bigrams_all, top_n=10)
    print(f"Keywords for '{page_title}': {keywords}")

else:
    print(document)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Keywords for 'Amitabh Bachan': ['the filmfare', 'the filmfare', 'the filmfare', 'the filmfare', 'the filmfare', 'the filmfare', 'with kapoor', 'filmfare with', 'bollywood in', 'with filmfare']


In [ ]:
import wikipedia
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.util import ngrams
from string import punctuation
import re
from collections import Counter
import numpy as np

# Download required NLTK data
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('stopwords')

class KeywordExtractor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.punctuation = set(punctuation)

    def fetch_wikipedia_content(self, title):
        """
        Fetch content from Wikipedia for a given title
        """
        try:
            page = wikipedia.page(title)
            return page.content
        except wikipedia.exceptions.DisambiguationError as e:
            return wikipedia.page(e.options[0]).content
        except wikipedia.exceptions.PageError:
            return None

    def split_into_sentences(self, text):
        """
        Split text into sentences using NLTK
        """
        return sent_tokenize(text)

    def clean_text(self, text):
        """
        Clean and preprocess text
        """
        # Convert to lowercase
        text = text.lower()

        # Remove special characters and digits
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)

        # Remove extra whitespace
        text = ' '.join(text.split())

        # Remove stopwords
        words = word_tokenize(text)
        words = [word for word in words if word not in self.stop_words]

        return ' '.join(words)

    def generate_ngrams(self, text, n_range=(4, 4)):
        """
        Generate n-grams from text
        """
        tokens = word_tokenize(text)
        all_ngrams = []

        for n in range(n_range[0], n_range[1] + 1):
            n_grams = list(ngrams(tokens, n))
            all_ngrams.extend([' '.join(gram) for gram in n_grams])

        return all_ngrams

    def extract_noun_phrases(self, text):
        """
        Extract noun phrases from text using POS tagging
        """
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        noun_phrases = []

        # Pattern for noun phrases (simple version: adjective? + noun+)
        current_phrase = []
        for word, tag in tagged:
            if tag.startswith(('JJ', 'NN')):  # Adjectives and nouns
                current_phrase.append(word)
            else:
                if current_phrase:
                    noun_phrases.append(' '.join(current_phrase))
                    current_phrase = []

        if current_phrase:  # Add the last phrase if exists
            noun_phrases.append(' '.join(current_phrase))

        return noun_phrases

    def max_sum_similarity(self, candidates, doc_embedding, top_n=30, nr_candidates=10):
        """
        Select keywords using max sum similarity
        """
        # Convert candidates to embeddings (using simple TF-IDF approach for demonstration)
        from sklearn.feature_extraction.text import TfidfVectorizer

        vectorizer = TfidfVectorizer()
        candidate_embeddings = vectorizer.fit_transform(candidates).toarray()
        doc_embedding = vectorizer.transform([doc_embedding]).toarray()[0]

        # Calculate similarities
        similarities = np.dot(candidate_embeddings, doc_embedding)

        # Get top candidates based on similarity
        sorted_indices = np.argsort(similarities)[::-1]
        top_candidates = [candidates[idx] for idx in sorted_indices[:nr_candidates]]

        # Select diverse set of keywords
        selected_keywords = []
        for _ in range(top_n):
            if not top_candidates:
                break

            # Select the candidate with highest similarity to document
            selected = top_candidates[0]
            selected_keywords.append(selected)

            # Remove selected candidate and similar candidates
            top_candidates = top_candidates[1:]

        return selected_keywords

    def extract_keywords(self, title, top_n=30):
        """
        Main function to extract keywords from Wikipedia article
        """
        # Fetch content
        content = self.fetch_wikipedia_content(title)
        if not content:
            return []

        # Split into sentences
        sentences = self.split_into_sentences(content)

        # Clean text
        cleaned_text = self.clean_text(' '.join(sentences  ))

        # Generate n-grams
        all_ngrams = self.generate_ngrams(cleaned_text)

        # Extract noun phrases
        noun_phrases = self.extract_noun_phrases(cleaned_text)

        # Combine n-grams and noun phrases
        candidates = list(set(all_ngrams ))

        # Select keywords using max sum similarity
        keywords = self.max_sum_similarity(candidates, cleaned_text, top_n=top_n)

        return keywords
# Example usage
if __name__ == "__main__":
    extractor = KeywordExtractor()
    keywords = extractor.extract_keywords("Amitabh  Bachan ", top_n=30)
    print("Keywords:", keywords)

Keywords: ['bachchan first national film', 'film award best actor', 'amitabh bachchan biggest film', 'bachchan started amitabh bachchan', 'actor award role film', 'bachchan first filmfare award', 'career bachchan made film', 'award role film year', 'also year bachchan starred', 'filmfare awards amitabh bachchan']


In [ ]:
def fetch_wikipedia_content(topic):
  try:
    page = wikipedia.page(topic)
    return page.content
  except Exception as e:
    print(f"Error fetching Wikipedia content: {e}")
  return None

In [ ]:
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

class TextCleaner:
    def __init__(self, language='english'):
        # Download required NLTK data
        nltk.download('punkt')
        nltk.download('stopwords')
        nltk.download('wordnet')

        self.stop_words = set(stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = PorterStemmer()

    def remove_html_tags(self, text):
        """Remove HTML tags from text"""
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)

    def remove_urls(self, text):
        """Remove URLs from text"""
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        return url_pattern.sub('', text)

    def remove_emoji(self, text):
        """Remove emojis from text"""
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            "]+", flags=re.UNICODE)
        return emoji_pattern.sub('', text)



    def remove_special_characters(self, text, remove_digits=True):
        """Remove special characters and optionally digits from text"""
        pattern = r'[^a-zA-Z\s]' if remove_digits else r'[^a-zA-Z0-9\s]'
        return re.sub(pattern, '', text)

    def remove_extra_whitespace(self, text):
        """Remove extra whitespace from text"""
        return ' '.join(text.split())

    def remove_stopwords(self, text):
        """Remove stopwords from text"""
        words = word_tokenize(text)
        return ' '.join([word for word in words if word.lower() not in self.stop_words])

    def lemmatize_text(self, text):
        """Lemmatize text"""
        words = word_tokenize(text)
        return ' '.join([self.lemmatizer.lemmatize(word) for word in words])

    def stem_text(self, text):
        """Stem text"""
        words = word_tokenize(text)
        return ' '.join([self.stemmer.stem(word) for word in words])

    def remove_punctuation(self, text):
        """Remove punctuation from text"""
        translator = str.maketrans('', '', string.punctuation)
        return text.translate(translator)

    def convert_lowercase(self, text):
        """Convert text to lowercase"""
        return text.lower()

    def clean_text(self, text,
                  lowercase=True,
                  remove_html=True,
                  remove_url=True,
                  remove_emojis=True,
                  expand_contractions=True,
                  remove_accents=True,
                  remove_special_chars=True,
                  remove_digits=True,
                  remove_punct=True,
                  remove_stops=True,
                  lemmatize=True,
                  stem=False,
                  extra_whitespace=True):
        """
        Main function to clean text with configurable options

        Args:
            text (str): Input text to clean
            lowercase (bool): Convert to lowercase
            remove_html (bool): Remove HTML tags
            remove_url (bool): Remove URLs
            remove_emojis (bool): Remove emojis
            expand_contractions (bool): Expand contractions
            remove_accents (bool): Remove accented characters
            remove_special_chars (bool): Remove special characters
            remove_digits (bool): Remove digits
            remove_punct (bool): Remove punctuation
            remove_stops (bool): Remove stopwords
            lemmatize (bool): Lemmatize text
            stem (bool): Stem text (not recommended with lemmatization)
            extra_whitespace (bool): Remove extra whitespace

        Returns:
            str: Cleaned text
        """
        if not isinstance(text, str):
            return ""

        # Apply cleaning operations based on parameters
        if remove_html:
            text = self.remove_html_tags(text)

        if remove_url:
            text = self.remove_urls(text)

        if remove_emojis:
            text = self.remove_emoji(text)


        if lowercase:
            text = self.convert_lowercase(text)


        if remove_special_chars:
            text = self.remove_special_characters(text, remove_digits)

        if remove_punct:
            text = self.remove_punctuation(text)

        if remove_stops:
            text = self.remove_stopwords(text)

        if lemmatize:
            text = self.lemmatize_text(text)

        if stem:
            text = self.stem_text(text)

        if extra_whitespace:
            text = self.remove_extra_whitespace(text)

        return text

# Example usage
if __name__ == "__main__":
    # Initialize the cleaner
     cleaner = TextCleaner()
     text = """
    <p>Hey! This is a sample text 😊 with HTML tags, emojis, and URLs https://example.com.
    It also has some números (numbers) and "special" chars!!
    Let's see how it's cleaned... We're testing contractions here.</p>
    """

    # Clean the text with default settings
     cleaned_text = cleaner.clean_text(text)
     print("Original text:", text)
     print("\nCleaned text:", cleaned_text)

    # Clean the text with custom settings
     custom_cleaned = cleaner.clean_text(
        text,
        lowercase=True,
        remove_html=True,
        remove_url=True,
        remove_emojis=True,
        expand_contractions=True,
        remove_accents=True,
        remove_special_chars=True,
        remove_digits=True,
        remove_punct=True,
        remove_stops=False,  # Keep stopwords
        lemmatize=False,     # No lemmatization
        stem=False,          # No stemming
        extra_whitespace=True
    )
     print("\nCustom cleaned text:", custom_cleaned)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original text: 
    <p>Hey! This is a sample text 😊 with HTML tags, emojis, and URLs https://example.com.
    It also has some números (numbers) and "special" chars!!
    Let's see how it's cleaned... We're testing contractions here.</p>
    

Cleaned text: hey sample text html tag emojis url also nmeros number special char let see cleaned testing contraction

Custom cleaned text: hey this is a sample text with html tags emojis and urls it also has some nmeros numbers and special chars lets see how its cleaned were testing contractions here


In [ ]:
import numpy as np
from collections import Counter, defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramAssocMeasures
from nltk.util import ngrams
import math
from typing import List, Dict, Tuple
import pandas as pd

class PMICalculator:
    def __init__(self, window_size: int = 5):
        """
        Initialize PMI Calculator

        Args:
            window_size (int): Size of the context window for co-occurrence
        """
        self.window_size = window_size
        self.word_freq = Counter()
        self.pair_freq = defaultdict(Counter)
        self.total_words = 0
        self.vocab_size = 0

    def preprocess_text(self, text: str) -> List[str]:
        """
        Preprocess text by tokenizing and converting to lowercase

        Args:
            text (str): Input text

        Returns:
            List[str]: List of preprocessed tokens
        """
        # Download required NLTK data if not already present
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

        # Tokenize and convert to lowercase
        tokens = word_tokenize(text.lower())
        return tokens

    def compute_co_occurrence(self, tokens: List[str]) -> None:
        """
        Compute word frequencies and co-occurrence frequencies

        Args:
            tokens (List[str]): List of preprocessed tokens
        """
        self.total_words = len(tokens)
        self.word_freq.update(tokens)
        self.vocab_size = len(self.word_freq)

        # Compute co-occurrence frequencies within window
        for i, word in enumerate(tokens):
            # Define context window
            start = max(0, i - self.window_size)
            end = min(len(tokens), i + self.window_size + 1)

            # Update co-occurrence frequencies
            context = tokens[start:i] + tokens[i+1:end]
            self.pair_freq[word].update(context)

    def calculate_pmi(self, word1: str, word2: str, smooth: bool = True) -> float:
        """
        Calculate PMI for a word pair

        Args:
            word1 (str): First word
            word2 (str): Second word
            smooth (bool): Whether to apply smoothing

        Returns:
            float: PMI value
        """
        # Get individual word probabilities
        p_w1 = self.word_freq[word1] / self.total_words
        p_w2 = self.word_freq[word2] / self.total_words

        # Get joint probability
        joint_count = self.pair_freq[word1][word2] + self.pair_freq[word2][word1]
        total_pairs = sum(sum(counter.values()) for counter in self.pair_freq.values()) / 2
        p_joint = joint_count / total_pairs

        # Apply smoothing if requested
        if smooth:
            # Add-one smoothing
            p_w1 = (self.word_freq[word1] + 1) / (self.total_words + self.vocab_size)
            p_w2 = (self.word_freq[word2] + 1) / (self.total_words + self.vocab_size)
            p_joint = (joint_count + 1) / (total_pairs + self.vocab_size)

        # Calculate PMI
        if p_joint > 0 and p_w1 > 0 and p_w2 > 0:
            pmi = math.log2(p_joint / (p_w1 * p_w2))
        else:
            pmi = float('-inf')

        return pmi

    def calculate_npmi(self, word1: str, word2: str) -> float:
        """
        Calculate Normalized PMI for a word pair

        Args:
            word1 (str): First word
            word2 (str): Second word

        Returns:
            float: NPMI value between -1 and 1
        """
        pmi = self.calculate_pmi(word1, word2)

        if pmi == float('-inf'):
            return -1.0

        # Get joint probability for normalization
        joint_count = self.pair_freq[word1][word2] + self.pair_freq[word2][word1]
        total_pairs = sum(sum(counter.values()) for counter in self.pair_freq.values()) / 2
        p_joint = joint_count / total_pairs

        if p_joint > 0:
            npmi = pmi / (-math.log2(p_joint))
        else:
            npmi = -1.0

        return npmi

    def get_top_collocations(self, n: int = 50) -> List[Tuple[Tuple[str, str], float]]:
        """
        Get top n word pairs by PMI score

        Args:
            n (int): Number of top collocations to return

        Returns:
            List[Tuple[Tuple[str, str], float]]: List of word pairs and their PMI scores
        """
        pmi_scores = []

        # Calculate PMI for all word pairs
        for word1 in self.word_freq:
            for word2 in self.pair_freq[word1]:
                if word1 < word2:  # Avoid duplicates
                    pmi = self.calculate_pmi(word1, word2)
                    if pmi != float('-inf'):
                        pmi_scores.append(((word1, word2), pmi))

        # Sort by PMI score
        pmi_scores.sort(key=lambda x: x[1], reverse=True)
        return pmi_scores[:n]

    def analyze_text(self, text: str) -> pd.DataFrame:
        """
        Analyze text and return PMI statistics

        Args:
            text (str): Input text

        Returns:
            pd.DataFrame: DataFrame containing word pairs and their PMI/NPMI scores
        """
        # Preprocess text and compute frequencies
        tokens = self.preprocess_text(text)
        self.compute_co_occurrence(tokens)

        # Get top collocations
        collocations = self.get_top_collocations(n=50)

        # Create DataFrame with results
        results = []
        for (word1, word2), pmi in collocations:
            npmi = self.calculate_npmi(word1, word2)
            results.append({
                'Word 1': word1,
                'Word 2': word2,
                'PMI': round(pmi, 3),
                'NPMI': round(npmi, 3),
                'Co-occurrences': self.pair_freq[word1][word2] + self.pair_freq[word2][word1]
            })

        return pd.DataFrame(results)

# Example usage
if __name__ == "__main__":
    # Sample text
    cleaner = TextCleaner()
     # Initialize and use PMI calculator
    pmi_calc = PMICalculator(window_size=5)
    cleanedtext = cleaner.clean_text(fetch_wikipedia_content ("Amitabh  Bachchan "))
    results_df = pmi_calc.analyze_text(cleanedtext)
    print("\nTop word associations by PMI score:")
   # cleanedtext = cleaner.clean_text(results_df )
    print(results_df)

    # Example of specific word pair analysis
    word1, word2 = "chote ", "miyan"
    pmi = pmi_calc.calculate_pmi(word1, word2)
    npmi = pmi_calc.calculate_npmi(word1, word2)
    print(f"\nPMI between '{word1}' and '{word2}': {pmi:.3f}")
    print(f"NPMI between '{word1}' and '{word2}': {npmi:.3f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Top word associations by PMI score:
                 Word 1       Word 2     PMI   NPMI  Co-occurrences
0           lokhandwala     shootout  10.684  0.938               8
1                africa      eastern  10.491  0.888               6
2                africa    mauritius  10.491  0.888               6
3                jumped        table  10.491  0.888               6
4                others        south  10.421  0.841               4
5                awadhi        hindu  10.421  0.841               4
6                awadhi     kayastha  10.421  0.841               4
7                awadhi       fluent  10.421  0.841               4
8            faisalabad       punjab  10.421  0.841               4
9            revolution     zindabad  10.421  0.841               4
10               phrase   revolution  10.421  0.841               4
11             inquilab   revolution  10.421  0.841               4
12           infatuated      parwana  10.421  0.841               4
13  lovertu

In [40]:
import wikipedia
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from typing import List, Tuple, Dict
import pandas as pd
from collections import Counter

class AdvancedKeywordExtractor:
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        Initialize the keyword extractor with specified transformer model
        """
        # Download required NLTK data
        nltk.download('punkt')
        nltk.download('stopwords')

        # Initialize SentenceTransformer model
        self.model = SentenceTransformer(model_name)
        self.stop_words = set(stopwords.words('english'))

    def fetch_wikipedia_content(self, topic: str) -> str:
        """
        Fetch content from Wikipedia for a given topic
        """
        try:
            page = wikipedia.page(topic)
            return page.content
        except Exception as e:
            print(f"Error fetching Wikipedia content: {e}")
            return ""

    def split_into_sentences(self, text: str) -> List[str]:
        """
        Split text into sentences and filter out short ones
        """
        sentences = sent_tokenize(text)
        # Filter out very short sentences (likely noise)
        return [s for s in sentences if len(s.split()) > 3]

    def clean_text(self, text: str) -> str:
        """
        Clean and preprocess text
        """
        # Convert to lowercase
        text = text.lower()

        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', '', text)

        # Remove special characters and digits
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)

        # Remove extra whitespace
        text = ' '.join(text.split())

        # Remove stopwords
        words = word_tokenize(text)
        words = [word for word in words if word not in self.stop_words]

        return ' '.join(words)

    def generate_ngrams(self, text: str, n_range: Tuple[int, int] = (2, 2)) -> List[str]:
        """
        Generate n-grams from text within specified range
        """
        tokens = word_tokenize(text)
        all_ngrams = []

        for n in range(n_range[0], n_range[1] + 1):
            n_grams = list(ngrams(tokens, n))
            all_ngrams.extend([' '.join(gram) for gram in n_grams])

        return all_ngrams

    def cluster_sentences(self, sentences: List[str], n_clusters: int = 5) -> Dict[int, List[str]]:
        """
        Cluster sentences using SentenceTransformer embeddings and KMeans
        """
        # Generate embeddings
        embeddings = self.model.encode(sentences)

        # Perform clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_assignments = kmeans.fit_predict(embeddings)

        # Group sentences by cluster
        clusters = {}
        for i, cluster in enumerate(cluster_assignments):
            if cluster not in clusters:
                clusters[cluster] = []
            clusters[cluster].append(sentences[i])

        return clusters

    def max_sum_similarity(self, candidates: List[str], top_n: int = 10) -> List[str]:
        """
        Select diverse keywords using max sum similarity
        """
        # Generate embeddings for candidates
        candidate_embeddings = self.model.encode(candidates)

        # Calculate similarity matrix
        similarities = cosine_similarity(candidate_embeddings)

        # Initialize selected indices and similarity sum
        selected_indices = []

        # Select keywords iteratively
        while len(selected_indices) < top_n:
            # Calculate similarity sum for each candidate
            if not selected_indices:
                # For first selection, choose the most central candidate
                similarity_sum = similarities.sum(axis=1)
                selected_idx = similarity_sum.argmax()
            else:
                # Calculate marginal similarity gain for remaining candidates
                remaining_indices = list(set(range(len(candidates))) - set(selected_indices))
                marginal_gains = np.zeros(len(remaining_indices))

                for i, idx in enumerate(remaining_indices):
                    marginal_gains[i] = sum(similarities[idx, selected_indices])

                # Select candidate with minimum marginal similarity (maximum diversity)
                selected_idx = remaining_indices[marginal_gains.argmin()]

            selected_indices.append(selected_idx)

        return [candidates[idx] for idx in selected_indices]

    def extract_keywords_from_cluster(self, sentences: List[str], top_n: int = 3) -> List[str]:
        """
        Extract keywords from a cluster of sentences
        """
        # Clean and preprocess sentences
        cleaned_sentences = [self.clean_text(sent) for sent in sentences]

        # Generate n-grams
        all_candidates = []
        for sent in cleaned_sentences:
            all_candidates.extend(self.generate_ngrams(sent))

        # Filter candidates by frequency
        candidate_freq = Counter(all_candidates)
        candidates = [cand for cand, freq in candidate_freq.items() if freq > 1]

        if not candidates:
            return []

        # Select diverse keywords using max sum similarity
        return self.max_sum_similarity(candidates, top_n=min(top_n, len(candidates)))

    def extract_keywords(self, topic: str, n_clusters: int = 10, keywords_per_cluster: int = 10) -> Dict:
        """
        Main function to extract keywords from a Wikipedia topic using clustering
        """
        # Fetch content
        content = self.fetch_wikipedia_content(topic)
        if not content:
            return {}

        # Split into sentences
        sentences = self.split_into_sentences(content)

        # Cluster sentences
        clusters = self.cluster_sentences(sentences, n_clusters=n_clusters)

        # Extract keywords from each cluster
        keywords_by_cluster = {}
        for cluster_id, cluster_sentences in clusters.items():
            keywords = self.extract_keywords_from_cluster(
                cluster_sentences,
                top_n=keywords_per_cluster
            )
            keywords_by_cluster[f"Cluster_{cluster_id}"] = {
                'keywords': keywords,
                'sample_sentences': cluster_sentences[:5]  # Store sample sentences for context
            }

        return keywords_by_cluster

# Example usage
if __name__ == "__main__":
    # Initialize the extractor
    extractor = AdvancedKeywordExtractor()

    # Extract keywords for a topic
    topic = "Amitabh Bachchan "
    results = extractor.extract_keywords(
        topic,
        n_clusters=5,
        keywords_per_cluster=5)

    # Print results
    print(f"\nKeywords extracted from '{topic}':")
    for cluster, data in results.items():
        print(f"\n{cluster}:")
        print("Keywords:", ", ".join(data['keywords']))
        print("\nSample sentences from this cluster:")
        for i, sent in enumerate(data['sample_sentences'], 1):
            print(f"{i}. {sent[:500]}...")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Keywords extracted from 'Amitabh Bachchan ':

Cluster_1:
Keywords: rai bachchan, millennium online, social causes, stage screen, poet harivansh

Sample sentences from this cluster:
1. Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema....
2. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B....
3. Amitabh Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hindi poet Harivansh Rai Bachchan and his wife, the social activist Teji Bachchan....
4. Bachchan also made an appearance in a Hollywood film, The Great Gatsby (2013), in which he played a non-Indian Jewish character....
5. In addition to acting, Bachchan has worked as a playback singer, film producer, and television presenter....

Cluster_4:
Keywords: filmfare award, officer legion, considered one, muqaddar ka, be